In [ ]:
!pip install openai pandas aiohttp tqdm

In [3]:
# --- Step 3: Setup ---
import os
import base64
import json
import asyncio
import pandas as pd
from pathlib import Path
from openai import AsyncOpenAI
import time
import nest_asyncio

# Patch asyncio for Jupyter/Colab
nest_asyncio.apply()

# GitHub token
os.environ["GITHUB_TOKEN"] = ""

# Instantiate client with GitHub endpoint
client = AsyncOpenAI(
    base_url="https://models.github.ai/inference",
    api_key=os.environ["GITHUB_TOKEN"],
)

MODEL = "openai/gpt-4o-mini"

# Rate limits
RPM = 15        # requests per minute
RPD = 150       # requests per day
CONCURRENT = 5  # max concurrent requests

# --- Step 4: Utils & G-VEval style prompt ---
def encode_image(image_path: Path):
    with open(image_path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

async def gveval_score_async(prompt_text: str, image_path: Path, meta_caption: str):
    image_b64 = encode_image(image_path)

    system_prompt = """You are a caption evaluation module following G-VEval from AAAI 2025.
You will be given:
- A reference caption.
- A candidate caption (meta-caption) generated from the image.
- The image itself.

You must produce:
- Four scores (Accuracy, Completeness, Conciseness, Relevance), each from 0 to 100.
- An overall score from 0 to 100.
- A short reasoning explaining the main strengths & weaknesses.

Return strictly JSON, exactly this structure:

{
  "accuracy": <int 0-100>,
  "completeness": <int 0-100>,
  "conciseness": <int 0-100>,
  "relevance": <int 0-100>,
  "overall": <int 0-100>,
  "reason": "<short explanation>"
}"""

    user_prompt = f"Reference: {prompt_text}\nCandidate: {meta_caption}"

    resp = await client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": user_prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
                ],
            },
        ],
        temperature=0,
        max_tokens=400,
    )

    return resp.choices[0].message.content

def safe_append_jsonl(out_path: str, record: dict):
    """Append record to jsonl safely (flush/sync) without overwriting previous content."""
    with open(out_path, "a", encoding="utf-8") as f:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")
        f.flush()
        os.fsync(f.fileno())

async def process_triple(triple, out_path: str):
    img_name, prompt, meta_caption, img_path = triple
    try:
        json_result = await gveval_score_async(prompt, img_path, meta_caption)
        record = {
            "image_name": img_name,
            "result": json.loads(json_result) if isinstance(json_result, str) else json_result
        }
        safe_append_jsonl(out_path, record)
        print(f"Saved: {img_name}")
        return record
    except Exception as e:
        print(f"Error for {img_name}: {e}")
        return None

async def run_eval(triples, out_path: str):
    sem = asyncio.Semaphore(CONCURRENT)
    start_time = time.time()
    completed = 0
    results = []

    total = len(triples)
    for i, triple in enumerate(triples, start=1):
        async with sem:
            # enforce per-minute limit
            elapsed = time.time() - start_time
            if completed >= RPM and elapsed < 60:
                await asyncio.sleep(60 - elapsed)
                completed = 0
                start_time = time.time()
            completed += 1
            rec = await process_triple(triple, out_path)
            if rec:
                results.append(rec)
            # verbose progress
            print(f"[{i}/{total}] Processed image: {triple[0]} | Total saved: {len(results)}")
            if len(results) >= RPD:
                print("Hit daily cap, stopping for today.")
                break
    return results

# --- Step 5: Load triples ---
def load_triples(prompt_csv: str, meta_csv: str, image_folder: str):
    df_prompts = pd.read_csv(prompt_csv)
    df_meta = pd.read_csv(meta_csv)

    # strip spaces from headers just in case
    df_prompts.columns = df_prompts.columns.str.strip()
    df_meta.columns = df_meta.columns.str.strip()

    triples = []
    for _, row in df_meta.iterrows():
        img_name = str(row["image_name"]).strip()
        meta_caption = row.get("Meta Caption", "")

        if not isinstance(meta_caption, str) or meta_caption.strip() == "":
            continue

        # find reference caption from df_prompts (matching image_name)
        prompt_match = df_prompts.loc[df_prompts["image_name"] == img_name, "mscoco_caption"].values
        prompt = prompt_match[0] if len(prompt_match) > 0 else ""

        # find image file
        img_path = None
        for ext in [".png", ".jpg", ".jpeg"]:
            candidate = Path(image_folder) / f"{img_name}{ext}"
            if candidate.exists():
                img_path = candidate
                break
        if img_path is None:
            continue

        triples.append((img_name, prompt, meta_caption, img_path))

    return triples

# --- Step 6: Run for SDXL model ---
base = "/content/drive/MyDrive/gveval"
prompt_csv = f"{base}/mscoco_captions.csv"
meta_csv = f"{base}/fluxdev_meta.csv"
image_folder = f"{base}/fluxdev"
out_file = f"{base}/mscoco_fluxdev_gveval.jsonl"

# Prepare data
triples = load_triples(prompt_csv, meta_csv, image_folder)
print(f"Loaded {len(triples)} triples.")

# Load already processed IDs and **filter triples before sending to GPT**
done_ids = set()
if Path(out_file).exists():
    with open(out_file, "r", encoding="utf-8") as f:
        for line in f:
            try:
                rec = json.loads(line)
                done_ids.add(rec["image_name"])
            except:
                pass
print(f"Already done: {len(done_ids)}")

# Filter out already processed triples
triples_to_process = [t for t in triples if t[0] not in done_ids]
print(f"Remaining images to process: {len(triples_to_process)}")

# Run evaluation (Colab-safe)
loop = asyncio.get_event_loop()
results = loop.run_until_complete(run_eval(triples_to_process, out_file))
print("Evaluation completed.")


Loaded 200 triples.
Already done: 119
Remaining images to process: 81
Saved: COCO_train2014_000000330575
[1/81] Processed image: COCO_train2014_000000330575 | Total saved: 1
Saved: COCO_train2014_000000335717
[2/81] Processed image: COCO_train2014_000000335717 | Total saved: 2
Saved: COCO_train2014_000000336003
[3/81] Processed image: COCO_train2014_000000336003 | Total saved: 3
Saved: COCO_train2014_000000337585
[4/81] Processed image: COCO_train2014_000000337585 | Total saved: 4
Saved: COCO_train2014_000000339130
[5/81] Processed image: COCO_train2014_000000339130 | Total saved: 5
Saved: COCO_train2014_000000346726
[6/81] Processed image: COCO_train2014_000000346726 | Total saved: 6
Error for COCO_train2014_000000351298: Error code: 400 - {'error': {'inner_error': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_results': {'sexual': {'filtered': True, 'severity': 'medium'}, 'violence': {'filtered': False, 'severity': 'safe'}, 'hate': {'filtered': False, 'severity': 'safe'}, '